# Aula 7

# "Data Pipelines" - Parte 3

# Dados estruturados usando `feature_column`

### Eduardo Lobo Lustosa Cabral

## 1. Objetivos

Apresentar o módulo `feature_column` do TensorFlow para realizar processamento de dados estruturados.

Apresentar a camada tipo **`DenseFeatures`** do Keras, que permite incluir o processamento dos dados dentro da rede.

Exemplos de "data pipeline" completo para dados estruturados.

### Importa principais biblitecas

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile

## 2. Transformação de dados com o TensorFlow

Com vimos, para processar e carregar dados de forma eficiente para treinar e utilizar uma RNA deve-se criar um "data pipeline". 

Os **Data pipelines** realizam três etapas de operação com os dados (Extrair, Transformar e Carregar):

- **Extrair** $\to$ carrega dados originais do local onde se encontram e traz para o nosso ambiente de computação;
- **Transformar** $\to$ processa os dados para serem colocados em formatos adequados que podem ser usados por uma RNA;
- **Carregar** $\to$ alimenta a RNA com dados durante o seu treinamento ou para realizar previsões.

O pré-processamento de dados (transformação) durante a fase de desenvolvimento de uma nova aplicação é realizado pelo desenvolvedor e pode ser realizado separadamente do treinamento da rede. Porém, na etapa de utilização da rede por outros usuários, o pré-processamento dos dados deve ser realizado de forma automática. Assim, a melhor forma é incluir o pré-processamento dentro da rede.

Como no Keras qualquer operação realizada por uma RNA deve ser uma camada, o módulo **`feature_column`** do TensorFlow permite incluir camadas em uma RNA para transformar dados (https://www.tensorflow.org/api_docs/python/tf/feature_column).

O módulo `feature_column` do TensorFlow serve para realizar transformações de dados estruturados $\to$ pode-se fazer praticamente qualquer tipo de transformação usando o módulo `feature_column`. 

Vamos ver algumas funcionalidades desse módulo para processar dados estruturados. 

O processamento de dados usado o módulo `feature_column` tem muitas vantagens em relação a usar outras forma, tais como, Pandas ou o Numpy. Entre essas vantagens tem-se:

1. Existem muitas ferramentas disponíveis;


2. Os dados processados podem ser facilmente transformados em um objeto `Dataset` do TensorFlow;


3. Como já visto, os objetos `Dataset` foram desenvolvidos para trabalhar em conjunto com o método `fit` do Keras e tem a capacidade de tornar o treinamento muito mais rápido em razão de otimizar o carregamento de lotes de dados na CPU/GPU.


4. Permite realizar em conjunto com uma camada tipo **DenseFeatrures** (https://www.tensorflow.org/api_docs/python/tf/keras/layers/DenseFeatures) a transformação dos dados dentro da rede, facilitando o seu uso por outros usuários.

A seguir veremos como realizar algumas dessas transformações.

## 3. Transformações em dados estruturados e textos

Para dados estruturados, onde cada característica consiste de uma coluna do conjunto de dados, as transformações são realizadas nas colunas.

As principais transformações realizadas em dados estruturados e textos são as seguintes:

- Colunas numéricas;
- Segmentação
- Categorias;
- Codificação "one-hot";
- Codificação "Hashed";
- Codificação "embedding";
- Cruzamento de colunas.

Na Aula 3 vimos o que significam quase todas essas transformações e como fazer usando o Pandas e Numpy.

O módulo `feature_column` do TensorFlow realiza todas essas transformações e muitas outras. A lista completa de transformações disponíveis pode ser vista em https://www.tensorflow.org/api_docs/python/tf/feature_column.

`Feature_columns` são objetos usados para alimentar RNAs de forma eficiente e quando é usado em dados estruturados todas as colunas de dados devem ser transformadas, não importando se forem dados numéricos ou não.

### 3.1 Conjunto de dados

Vamos utilizar um sub-conjunto dos dados do conjunto de dados **Wine Reviews**, disponível no Kaggle (https://www.kaggle.com/christopheiv/winemagdata130k). Esse conjunto de dados completo consiste de 130 mil avaliações de vinhos, que inclui variedade, região onde é produzido, vinícola, preço e descrição do vinho.

Para exemplificar as transformações realizadas com o módulo `feature_column`, vamos utilizar 1001 linhas desse conjunto de dados e as seguintes características:

- "country";
- "description";
- "points";
- "price";
- "province";
- "variety".

Esses dados estão em um arquivo tipo CSV. Cada linha representa um vinho e cada coluna descreve uma característica do vinho. O objetivo desses dados é prever o preço do vinho, assim, consiste de um problema de regressão.

####  Carregar conjunto de dados

Vamos usar o Pandas para carregar os dados de um arquivo CSV e carregá-lo em um objeto Dataset.

In [2]:
# Carrega dados em um DataFrame Pandas
df = pd.read_csv('wine_review_small.csv', index_col=0)

# Apresenta DataFrame
df

,country,description,points,price,province,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Pinot Noir
...,...,...,...,...,...,...
996,Italy,Here's a Syrah with bursting aromas of mature ...,88,14.0,Sicily & Sardinia,Syrah
997,Australia,Blended from a patchwork of old vineyards thro...,88,18.0,South Australia,Shiraz
998,US,"Rich in the mouth, this creamy and textural wi...",88,18.0,Oregon,Pinot Gris
999,US,"Creamy and textural, this brings on a nice mix...",88,17.0,Oregon,Riesling


- Observa-se que existem dados de 1001 vinhos, ou seja, existem 1001 exemplos.

Vamos verificar o tipo de dados e calcular as principais estatísticas dos dados de cada coluna do DataFrame para entender um pouco como são esses dados.

In [3]:
# Tipos de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 6 columns):
country        1000 non-null object
description    1001 non-null object
points         1001 non-null int64
price          944 non-null float64
province       1000 non-null object
variety        1001 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 54.7+ KB


In [5]:
# Principais estatísticas das colunas numéricas
df.describe()

,points,price
count,1001.000000,944.000000
mean,88.581419,37.337924
std,2.625569,47.302631
min,80.000000,7.000000
25%,87.000000,17.000000
50%,88.000000,27.000000
75%,90.000000,43.250000
max,100.000000,775.000000


#### Verificar colunas que possuem dados não existentes

A primeira etapa do processamento de dados é verificar se existem dados ausentes (NaN) e onde eles estão.

Para isso podemos criar uma função que procura dados ausentes em uma coluna, usando o método `isnull()`, e depois utilizá-la com o método `apply()` para procurar em todas as colunas. 

O código da célula abaixo realiza essa operação. 

In [7]:
# Cria função que calcula número de NaNs em uma coluna
def num_missing(x):
    return sum(x.isnull())

# Aplica função num_missing em todas as colunas
print("Valores ausentes por coluna")
print(df.apply(num_missing, axis=0))

Valores ausentes por coluna
country         1
description     0
points          0
price          57
province        1
variety         0
dtype: int64


#### Correção de erros

Vamos remover todas as linhas que possuem dados faltantes (NaN).

In [8]:
df = df.drop(df[pd.isnull(df.country)].index)

In [9]:
df = df.drop(df[pd.isnull(df.province)].index)

In [10]:
df = df.drop(df[pd.isnull(df.price)].index)
df

,country,description,points,price,province,variety
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Pinot Noir
5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Tempranillo-Merlot
...,...,...,...,...,...,...
996,Italy,Here's a Syrah with bursting aromas of mature ...,88,14.0,Sicily & Sardinia,Syrah
997,Australia,Blended from a patchwork of old vineyards thro...,88,18.0,South Australia,Shiraz
998,US,"Rich in the mouth, this creamy and textural wi...",88,18.0,Oregon,Pinot Gris
999,US,"Creamy and textural, this brings on a nice mix...",88,17.0,Oregon,Riesling


In [11]:
# Aplica função num_missing em todas as colunas
print("Valores ausentes por coluna")
print(df.apply(num_missing, axis=0))

Valores ausentes por coluna
country        0
description    0
points         0
price          0
province       0
variety        0
dtype: int64


### 3.2 Criar objeto Dataset

Para criar um pipeline de dados, a primeira etapa é criar um objeto Dataset a partir do DataFrame Pandas. 

Com um objeto Dataset é possível criar camadas tipo `feature_columns`, que servem como uma ponte entre os dados e a RNA. 

Lembre que se o arquivo de dados for muito grande, pode-se criar um Dataset carregando-o diretamente do arquivo no disco, lote por lote. Já vimos como fazer isso na Aula 5.

Na célula abaixo é definida uma função para criar um Dataset a partir do DataFrame Pandas. Nessa função são realizadas as seguintes operações:

1. Separação das saídas desejadas;
2. Embaralhamento aleatóriamente dos dados, se for desejado;
3. Geração de lotes de dados.

In [12]:
# Função para criar Dataset a partir de um DataFrame Pandas
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    # Cria cópia dpo Dataframe
    dataframe = dataframe.copy()
    
    # Separa saídas desejadas
    labels = dataframe.pop('price')
    
    # Cria Dataset
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    
    # Embaralha dados
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        
    # Gera lotes de dados
    ds = ds.batch(batch_size)
    
    return ds

Vamos usar a função `df_to_datset` para criar o Dataset a partir do DataFrame Pandas `df`.

In [13]:
# Define tamanho do lote
batch_size = 5

# Cria Dataset
ds = df_to_dataset(df, batch_size=batch_size)

Para verificar se o Dataset foi criado corretamente, vamos gerar um exemplo e observar suas características.

In [14]:
for feature_batch, label_batch in ds.take(1):
    print('Lista das características:', list(feature_batch.keys()))
    print('\nUm lote de country:', format(feature_batch['country']))
    print('\nUm lote de description:', format(feature_batch['description']))
    print('\nUm lote de points:', format(feature_batch['points']))
    print('\nUm lote de variety:', format(feature_batch['variety']))
    print('\nUm lote de saída (price):', format(label_batch ))

Lista das características: ['country', 'description', 'points', 'province', 'variety']

Um lote de country: [b'Italy' b'Italy' b'US' b'Austria' b'Italy']

Um lote de description: [b'At the first it was quite muted and subdued, but over a ten minute period it developed beautifully in the glass. The dark fruit is still dominant, but it is beginning to show some interesting tertiary aromas of earth, mushrooms and leather. Well balanced with a medium finish. Needs a few more years, but can be enjoyed now with a good deal of pleasure.'
 b'Made with 65% Sangiovese, 20% Merlot and 15% Cabernet Sauvignon, this has subtle aromas of black-skinned fruit and thyme. The easygoing palate delivers black cherry and cinnamon alongside smooth tannins.'
 b"Juicy plum, raspberry and pencil lead lead the way in this vineyard designate, a site that's 2,000 feet high. Tobacco and cedar meet a full-bodied hit of oak and puckering tannin, the wine still youthfully wrapped in its full-bodied boldness."
 b'Tende

- Observe que o Dataset retorna um dicionário de nomes das colunas do DataFrame, onde cada coluna representa uma característica e os seus valores se referem a cada vinho (uma linha de dados).

### 3.3 Lote de exemplo

Nos próximos itens vamos criar objetos `features_columns` que transformam os dados para uma forma que pode ser usada por uma RNA.

Os objetos `feature_columns` são inseridos na RNA usando uma camada `Densefeatures` e os resultados das transformações são as entradas das camadas da rede, sem a necessidade de pré-processamento dos dados.  

Para isso vamos usar um lote de dados de treinamento para exemplificar as transformações.

In [15]:
# Gera um lote de dados de entrada do conjunto de treinamento
example_batch = next(iter(ds))[0]

# Mostra lote gerado
print(example_batch)

{'country': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Italy', b'Germany', b'Australia', b'US', b'Chile'], dtype=object)>, 'description': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"The dynamic sisters that run this estate deliver a clean and territory-driven line of white wines. This easy Soave Classico opens with peach, pear, citrus and a touch of fragrant honeysuckle flower. It's pure and focused overall.",
       b'Just a touch of honey-lemon sweetness is enough to make this zesty, citrus-focused Riesling lip smackingly delicious. Fresh, green herb and lime notes reverberate through a strong finish. Drinks well now but should meld nicely over the next five years.',
       b"This deep brown wine smells like a damp, mossy cave. Then add complex rancio notes, plus maple syrup and molasses. It's full, round and harmonious, wonderfully rich yet without any sense of heaviness and long and bright on the finish.",
       b'Shows the briary, brambly character of Foothill

### 3.4 Visualização dos resultados das `feature_columns`

`Feature_columns` são objetos que representam uma camada de uma RNA. Assim, para poder visualizar os resultados das transformações é necessário criar uma rede neural com uma camada do tipo `DenseFeatures` que aplica a transformação definida para processar os dados.

In [17]:
# Importa módulo feature_column e a classe de camadas
from tensorflow import feature_column
from tensorflow.keras import layers

# Cria RNA de uma camada para processar dados e visualizar transformação definida
def rna_demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

- Usando essa `rna_demo` podemos visualizar claramente como cada coluna dos dados é transformada.


- Veremos mais detalhes das camadas tipo `DenseFeatures` quando formos criar uma RNA com esse tipo de camada.

### 3.5 Valores numéricos

Uma coluna numérica (https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) é a forma mais simples de um objeto `feature_column`. Ela é usada para representar valores numéricos. Quando essa coluna é usada, a RNA recebe os valores da coluna inalterados.

O método utilizado para fazer essa transformação é o `tf.feature_column.numeric_column()`. Nesse tipo de `feature_column` a RNA recebe os valores do Dataset sem nenhuma transformação.

Os dados de entrada de uma RNA devem ser normalizados, assim, devemos normalizar os valores de points. Quando calculamos as estaíticas dos dados numéricos desse conjunto de dados, vimos que o valor mínimo de "points" é 80 e o valor máximo é 100.

Vamos aplicar a seguinte normalização para a característica "points" de forma que os seus valores normalizados fiquem entre 0 e 1.

$$point_{norm} = \frac {point - point_{min}} {point_{max} - point_{min}}$$

Para normalizar dados numéricos temos que definir uma função que realiza essa tranformação e ao criar a coluna numérica devemos chamar essa função.

Para criar uma coluna numérica da coluna `points`, com valores normalizados, se faz o seguinte.

In [19]:
# Define função de normalização
def norm_points(col):
    min_points = 80
    max_points = 100
    return (col - min_points)/(max_points - min_points)

# Criação da coluna "points" normalizada
points_norm = tf.feature_column.numeric_column("points", normalizer_fn=norm_points)

# Apresenta resultados da transformação e a idade original
rna_demo(points_norm)

[[0.35]
 [0.55]
 [0.9 ]
 [0.25]
 [0.  ]]


### 3.6 Segmentação de valores

Frequentemente, não queremos fornecer um número diretamente à RNA, mas sim dividir seu valor em diferentes categorias com base em intervalos numéricos. Considere novamente a coluna de pontuação ("points), podemos dividi-la em vários segmentos usando uma coluna segmentada. Por exemplo, usando 4 intervalos:

    pointos < 85        --> [1, 0, 0, 0]
    85 <= pointos < 90  --> [0, 1, 0, 0]
    90 <= pointos < 95  --> [0, 0, 1, 0]
    pointos >= 95       --> [0, 0, 0, 1]

Nessa codificação cada faixa de pointos é reprentada por um vetor de 4 elementos, que representa a faixa de pontos de cada vinho. Os intervalos incluem o limite esquerdo e excluem o limite direito. 

O método utilizado para fazer essa transformação é o `tf.feature_column.bucketized_column()` (https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). 

O códico abaixo mostra como fazer essa transformação. Observe que antes de segmentar a pontuação temos que transformá-la em uma coluna numérica.

In [22]:
# Transforma idade em coluna numérica
points = tf.feature_column.numeric_column("points")

# Segmenta idade
points_seg = tf.feature_column.bucketized_column(points, boundaries=[85, 90, 95])

# Apresenta resultados da segmentação
rna_demo(points_seg)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


### 3.7 Váriaveis categóricas (codificação "one-hot")

Neste conjunto de dados, as características "country", "province" e "variety" são representadas por strings. Não podemos alimentar strings diretamente em uma RNA. Em vez disso, devemos primeiro mapeá-las para valores numéricos. 

Para transformar dados categóricos na forma de strings para números inteiros tem-se dois métodos:

- `tf.feature_column.categorical_column_with_vocabulary_list` (https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list); e
- `tf.feature_colums.categorical_column_with_vocabulary_file` (https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file). 

No primeiro método o vocabulário (valores) é passado como uma lista e no segundo é carregado de um arquivo.

Após mapear as categorias em números inteiros, elas devem ser codificados para vetores one-hot com dimensão igual ao número de categorias. 

Lembre que a codificação one-hot é muito usada em problemas de classificação multiclasse, onde as classes dos objetos em geral são especificadas ou por nomes ou números inteiros.
  
O método utilizado para realizar a codificação "one-hot" é `tf.feature_column.indicator_column` (https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column). O código a seguir mostra como realizar essa transformação para as características "country" e "province".

In [23]:
# Transforma coluna "country" strings em categorias
country = tf.feature_column.categorical_column_with_vocabulary_list('country', 
          vocabulary_list=df['country'].unique(), default_value=0)

# Transforma números inteiros em vetores one-hot
country_hot = tf.feature_column.indicator_column(country)

# Número de categorias e "country"
print('Número de categorias:', len(df.country.unique()))

# Apresenta resultados da codificação
rna_demo(country_hot)

Número de categorias: 18
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


- Observe que na transformação das strings em inteiros, o vocabulário foi passado pela própria coluna "country" do dataframe. Essa forma de passar o vocabulário facilita muito.

In [24]:
# Transforma coluna "country" strings em categorias
province = tf.feature_column.categorical_column_with_vocabulary_list('province', 
          vocabulary_list=df['province'].unique(), default_value=0)

# Transforma números inteiros em vetores one-hot
province_hot = tf.feature_column.indicator_column(province)

# Número de categorias e "country"
print('Número de categorias:', len(df.province.unique()))

# Apresenta resultados da codificação
rna_demo(province_hot)

Número de categorias: 95
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.

### 3.8 Codificação "hash"

Outra maneira de representar uma coluna categórica com um grande número de valores é usar o método `categorical_column_with_hash_bucket()`. 

Uma função "hash" é um algoritmo que mapeia dados de comprimento variável para dados de comprimento fixo. Para obter mais detalhes sobre essa forma de codificação ver o link: https://www.cs.cmu.edu/~adamchik/15-121/lectures/Hashing/hashing.html.

Nesse método é calculado um vetor "hash" das categorias e depois esse vetor é segmentado com o número de intervalos desejados. 

Esse método não exige fornecer a lista de vocabulário e pode-se escolher o número de intervalos ("hash_buckets"). Com isso pode-se ter um número de elementos significativamente menor do que o número de categorias reais para economizar espaço.

**Importante:**

- Ao escolher um número de segmentos ("hash_buckets") menor do que o número de categorias reais, estamos forçando as diferentes categorias para um conjunto menor de códigos. 

- Isso significa que duas categorias não relacionadas podem ser mapeadas com o mesmo código e, conseqüentemente, passam a significar a mesma coisa para a rede neural $\to$ nesse caso, pode haver colisões nas quais diferentes categorias são mapeadas para o mesmo intervalo. 

- Na prática, isso pode causar problemas e deve ser usado com cuidado.

A transformação "hash" é realizada com o método `tf.feature_column.categorical_column_with_hash_bucket()` da seguinte forma:

In [26]:
# Transformação hashed
province_hashed = tf.feature_column.categorical_column_with_hash_bucket(
      'province', hash_bucket_size=30)

# Para facilitar a visualização da codificação hashed devemos transformá-la em um vetor one-hot
print('Codificação hashed da coluna province')
rna_demo(tf.feature_column.indicator_column(province_hashed))

Codificação hashed da coluna province
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]


### 3.9 Cruzamento de características

Combinar caracteríticas, também conhecido como cruzamentos de características, permite expandir os dados de entrada de forma a criar novas características que sejam mais significativas para o problema.

Combinar caracteríticas também permite diminuir os dados de entrada ao usarmos o cruzamento de duas características no lugar de usar as duas, quando as duas características possuem informações redundantes.

Pode-se criar um cruzamento de características a partir dos seguintes tipos de dados:

- Qualquer dado categórico;
- Qualquer dado numérico;
- Qualquer dado segmentado.

Existem muitas formas de cruzar características. A forma utilizada depende do problema e do tipo de dados sendo cruzados. Algumas formas de cruzar caracteríticas são as seguintes:

1. Para dados numéricos pode-se multiplicar elemento por elemento;


2. Para dados numéricos pode-se ajustar um polinômio de grau "n" aos dados;


3. Para dados categóricos (ou segmentados) pode-se criar uma tabela 2D com as categorias de cada característica sendo as linhas e as colunas da tabela. Nesse caso, por exemplo, se tivermos uma caraterística com 5 classes e a outra com 10 classes, teremos no cruzamento das duas caraterísticas 50 categorias possíveis. 


#### Cruzamento de dados categóricos ou segmentados

Ao cruzar duas características categóricas ou segmentadas, em princípio criamos uma nova característica representada por um vetor cuja dimensão é igual ao produto dos números de categorias/segmentos das duas características. 

Na maior parte dos casos uma grade completa é tratável apenas para entradas com vocabulários limitados, assim, em vez de construir uma tabela completa (potencialmente enorme) ao criarmos uma característica cruzada podemos escolher o número de novas categorias.

A diminuição do número de categorias do cruzamento de duas características pode ser feita usando a codificação "hash". Porém, como vimos, usar a função "hash" limita o número de categorias, mas pode causar colisões de categorias, ou seja, vários cruzamentos de categorias acabarão no mesmo intervalo "hash".

O cruzamento de duas características é feito com o método `tf.feature_column.crossed_column` (https://www.tensorflow.org/api_docs/python/tf/feature_column/crossed_column). 

Esse método não cria uma tabela completa de todas as combinações possíveis. O resultado do cruzamento de características é processado por uma codificação Hash de forma a não criar a tabela completa de todas as combinações possíveis. Assim, pode-se escolher o número de elementos resultantes do cruzamento.

Como exemplo, vamos criar uma nova careterística cruzando as colunas "country" e "province".

Antes de realizar esse cruzamento vamos verificar o número de categorias dessas duas características.

In [27]:
print('Número de países:', len(df.country.unique()))
print('Numero de provincias:', len(df.province.unique()))

Número de países: 18
Numero de provincias: 95


Temos:

- Número de categorias de "country" = 18;
- Número de categorias de "province" = 95;

Portanto, a combinação dessas duas categorias sem perda nennhuma de informação gera uma nova característica com 18*95 = 1710 categorias. Porém vamos escolher somente 200 categorias no resultado do cruzamento.

In [29]:
# Transforma coluna "country" strings em categorias
country = tf.feature_column.categorical_column_with_vocabulary_list('country', 
          vocabulary_list=df['country'].unique(), default_value=0)

# Transforma coluna "country" strings em categorias
province = tf.feature_column.categorical_column_with_vocabulary_list('province', 
          vocabulary_list=df['province'].unique(), default_value=0)

# Cria nova caracterítica pelo cruzamento das categorias de contry e province
crossed_feature = tf.feature_column.crossed_column([country, province], hash_bucket_size=20)

# Processa a característica crossed_feature
print('Resultado do cruzamento')
rna_demo(tf.feature_column.indicator_column(crossed_feature))

Resultado do cruzamento


OverflowError: Python int too large to convert to C long

### 3.10 Codificação de texto (Embedding) 

Existem duas formas de codificar textos.

#### Codificação de texto com "one-hot"

A codificação de palavras (ou letras) de um texto pode ser realizada pelo método "one-hot". Porém, essa forma de codificação deve ser usada somente quando se tem um dicionário de palavras pequeno.


#### Codificação de texto com "embedding"

Suponha que em vez de ter apenas algumas categorias tenhamos milhares (ou mais). Por uma série de razões, na medida em que o número de categorias cresce, torna-se inviável treinar uma rede neural usando codificação "one-hot". 

Esse tipo de problema ocorre frequentemente em processamento de texto, onde pode-se ter um dicionário com milhares de palavras.

Nesses casos se usa a codificação "embedding" para superar essa limitação. 

Em vez de representar as categorias como um vetor de dimensão grande com um único elemento diferente de zero, a codificação "embedding" representa cada categoria por um vetor denso com dimensão menor do que o número de categorias, onde cada elemento contém um número real.

O tamanho do vetor na codificação "embedding" é um parâmetro que deve ser ajustado para o problema que se está resolvendo.

A codificação "embedding" é realizada com o método `tf.feature_column.embedding_column()` (https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column). 

Para exemplificar, vamos codificar a coluna "variety" para "one-hot" e "embedding", e analisar a diferença. A codificação "one-hot" é realizada somente para comparação e não precisa ser realizada para obter a codificação "embedding".

In [ ]:
# Número de categorias da coluna "variety"
print('Número de variedades:', len(df.variety.unique()))

# Transforma strings em números inteiros
variety = tf.feature_column.categorical_column_with_vocabulary_list('variety', 
           vocabulary_list=df['variety'].unique(), default_value=0)

# Transformação one-hot 
variety_hot = tf.feature_column.indicator_column(variety)

# Transformação "embedding"
variety_embedding = tf.feature_column.embedding_column(variety, dimension=10)

# Apresentação do resultado das codificações e da coluna conceito original
print('\nCodificação "one-hot"')
rna_demo(variety_hot)
print(' ')

print('Codificação "embedding"')
rna_demo(variety_embedding)
print(' ')

- Note que a entrada para transformação "embedding" são os dados categóricos obtidos na transformação com o método  `tf.feature_column.categorical_column_with_vocabulary_list()`.


- Como temos 130 categorias na característica "variety", então a codificação one-hot dessa característica gera vetores de 130 elementos para cada categoria.


- O argumento `dimension` é a dimensão desejada para os vetores de codificação "embedding", que nesse exemplo foi escolhido igual a 10.


**Importante:**

- A codificação "embedding" é aprendida durante o treinamento da RNA juntamente com os parâmetros das camadas da RNA.

- Nos vetores de codificação "embedding", os valores são números aleatórios usados para iniciar o processo de treinamento.

#### Comparação "one-hot" e "embedding"

Vejamos um exemplo de comparação de codificação "one-hot" e "embeding". Suponha que os exemplos de entrada consistam de uma única palavra de um conjunto de 81 palavras possíveis. Suponha ainda que temos 4 exemplos com as seguintes categorias:

    "dog"
    "spoon"
    "acissors"
    "guitar"

Nesse caso, a Figura 1 ilustra a codificação desses 4 exemplos usando "one-hot" e "embeding".

<br>
<img src="Fig_Aula3_OneHot_Embeding.png">
<center>Figura 1- Codificação one-hot e embedding (https://medium.com/ml-book)</center>
<br>

Na codificação "embedding" ao usar números reais, pode-se ter um vetor com um número de elementos muito menor do que na codificação "one-hot". Os valores que aparecem nos vetores "embedding" são números aleatórios usados para inicializar o treinamento. Os valores mais adequados para o problema são determinados durante o treinamento.

Quando um exemplo é processado, o método `tf.feature_column.categorical_column_with_vocabulary_list()` mapeia cada categoria (string) dos exemplos para um valor numérico entre 0 e número de categorias menos 1. Por exemplo, a categoria  “colher” é a de número 32. A partir dessa codificação inicial pode-se escolher codificar as categorias de duas formas diferentes:

- "One-hot". Nesse caso a codificação converte cada valor categórico numérico em um vetor de 81 elementos (porque nosso conjunto de dados possui 81 palavras), colocando 1 no elemento de índice igual ao valor categórico (0, 32, 79, 80) e 0 em todas as outras posições.


- "Embedding". Nesse exemplo a codificação "embeding" usa os valores categóricos numéricos (0, 32, 79, 80) como índices para criar vetores que contém 3 elementos.


Como os valores nos vetores "embeddings são atribuídos? As atribuições acontecem durante o treinamento, ou seja, o modelo aprende a melhor maneira de mapear seus valores categóricos numéricos de entrada para o valor do vetor de "embedding" para resolver o problema. 

A codificação "embedding" aumenta a capacidade de generalização da RNA, uma vez que um vetor "embedding" aprende novos relacionamentos entre as categorias a partir dos dados de treinamento.

Para permitir um melhor entendimento, por exemplo, no caso de palavras em um problema de tradução de texto, pode-se imaginar que a codificação "embedding" representa, por exemplo, o tipo de palavra (substantivo, verbo, pronome, adjetivo etc), o seu significado e a sua frequência de uso. Seria como um "dicionário digital" das palavras.

## 4. Pipeline para dados estruturados

Vimos como funcionam alguns tipos de `feature_columns` para dados estruturados. Vamos agora preparar os dados para serem usados por uma RNA e mostrar um exemplo completo, desde a transformação dos dados, passando pelo treinamento da RNA, até a utilização da rede com dados brutos.

### 4.1 Conjunto de dados

Para exemplificar, vamos utilizar um pequeno conjunto de dados de doenças de coração disponibilizado pela Cleveland Clinic Foundation for Heart Disease (https://archive.ics.uci.edu/ml/datasets/heart+Disease).

Esses dados estão em um arquivo tipo CSV. Cada linha representa um paciente e cada coluna descreve uma característica do paciente. O objetivo desses dados é prever se os pacientes apresentam ou não doença de coração, assim, consiste de um problema de classificação binária. 

Uma descrição dos dados desse conjunto é fornecida em https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names e um resumo segue abaixo. Note que existem dados numéricos e categóricos.


>Column| Description| Feature Type | Data Type
>------------|--------------------|----------------------|-----------------
>Age | Age in years | Numerical | integer
>Sex | (1 = male; 0 = female) | Categorical | integer
>CP | Chest pain type (0, 1, 2, 3, 4) | Categorical | integer
>Trestbpd | Resting blood pressure (in mm Hg on admission to the hospital) | Numerical | integer
>Chol | Serum cholestoral in mg/dl | Numerical | integer
>FBS | (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) | Categorical | integer
>RestECG | Resting electrocardiographic results (0, 1, 2) | Categorical | integer
>Thalach | Maximum heart rate achieved | Numerical | integer
>Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical | integer
>Oldpeak | ST depression induced by exercise relative to rest | Numerical | integer
>Slope | The slope of the peak exercise ST segment | Numerical | float
>CA | Number of major vessels (0-3) colored by flourosopy | Numerical | integer
>Thal | 3 = normal; 6 = fixed defect; 7 = reversable defect | Categorical | string
>Target | Diagnosis of heart disease (1 = true; 0 = false) | Classification | integer

####  Carregar conjunto de dados

Vamos usar o Pandas para carregar os dados de um arquivo CSV e carregá-lo em um objeto Dataset.

In [ ]:
# Carrega dados em um DataFrame Pandas
df = pd.read_csv('heart.csv')

# Apresenta DataFrame
df

- Observa-se que existem dados de 303 pacientes, ou seja, existem 303 exemplos.

Vamos verificar o tipo de dados e calcular as principais estatísticas dos dados de cada coluna do DataFrame para entender um pouco como são esses dados.

In [ ]:
# Tipos de dados
df.info()

In [ ]:
# principais estatísticas
df.describe()

#### Correção de erros

Esse conjunto de dados tem dois dados com problema na coluna 'thal'. Vamos identificar esses exemplos com problemas e corrigir.

In [ ]:
# Valores existentes na coluna thal
print(df['thal'].unique())

In [ ]:
# Identifica exemplos com erros
df.loc[df.thal=='1']

In [ ]:
# Identifica exemplos com erros
df.loc[df.thal=='2']

In [ ]:
df['thal'] = df['thal'].replace(['1'],'normal')
df['thal'] = df['thal'].replace(['2'],'reversible')

### 4.2 Divisão dos dados em conjuntos de treinamento e teste

Vamos dividir os dados em conjuntos de treinamento e teste. Observe que o conjunto de teste também vai ser usado como dados de validação.

In [ ]:
# Importa função para dividir dados da biblioteca ScikitLearn
from sklearn.model_selection import train_test_split

# Divisão dos dados
df_train, df_test = train_test_split(df, test_size=0.2)

# Apresenta número de exemplos em cada conjunto
print('Número de exemplos de treinamento:', len(df_train))
print('Número de exemplos de teste:', len(df_test))

### 4.3 Criar objetos Dataset

Para criar um pipeline de dados, a primeira etapa é criar um objeto Dataset a partir do DataFrame Pandas. 

Os dados em um objeto Dataset permite criar camadas tipo `feature_columns`, que servem como uma ponte entre os dados e a RNA. 

Lembre que se o arquivo de dados for muito grande, pode-se criar um Dataset carregando-o diretamente do arquivo no disco, lote por lote. Já vimos como fazer isso na Aula 4.

Na célula abaixo é definida uma função para criar um Dataset a partir do DataFrame Pandas. Nessa função são realizadas as seguintes operações:

1. Separação das saídas desejadas;
2. Embaralhamento aleatóriamente dos dados, se for desejado;
3. Geração de lotes de dados.

In [ ]:
# Função para criar Dataset a partir de um DataFrame Pandas
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    # Cria cópia dpo Dataframe
    dataframe = dataframe.copy()
    
    # Separa saídas desejadas
    labels = dataframe.pop('target')
    
    # Cria Dataset
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    
    # Embaralha dados
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        
    # Gera lotes de dados
    ds = ds.batch(batch_size)
    
    return ds

Vamos usar a função `df_to_datset` para criar os Datasets de treinamento e teste a partir dos DataFrames Pandas (`df_train` e `df_test`).

In [ ]:
# Define tamanho do lote
batch_size = 5

# Cria Dataset de treinamento
train_ds = df_to_dataset(df_train, batch_size=batch_size)

# Cria Dataset de teste
test_ds = df_to_dataset(df_test, shuffle=False, batch_size=batch_size)

- Observa-se que não foi realizado o embaralhamento dos dados na criação dos Datasets porque na divisão dos dados nos conjutnos de treinamento e teste, usando a função `train_test_split()` da biblioteca ScikitLearn essa operação já é relizada.

Para verificar se os Datasets de treinamento e teste foram criados corretamente, vamos gerar um exemplo e observar suas características.

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('Lista das características:', list(feature_batch.keys()))
    print('Um lote de idade:', format(feature_batch['age']))
    print('Um lote de sexo:', format(feature_batch['sex']))
    print('Um lote de cp:', format(feature_batch['cp']))
    print('Um lote de trestbps:', format(feature_batch['trestbps']))
    print('Um lote de chol:', format(feature_batch['chol']))
    print('Um lote de fbs:', format(feature_batch['fbs']))
    print('Um lote de restecg:', format(feature_batch['restecg']))  
    print('Um lote de thalach:', format(feature_batch['thalach']))   
    print('Um lote de oldpeak:', format(feature_batch['oldpeak']))
    print('Um lote de ca:', format(feature_batch['ca']))
    print('Um lote de thal:', format(feature_batch['thal']))    
    print('Um lote de saída:', format(label_batch ))

- Observe que o Dataset retorna um dicionário de nomes das colunas do DataFrame, onde cada coluna representa uma característica e os seus valores se referem a cada paciente (uma linha de dados).

### 4.4 Criar lote de exemplo

Nos próximos itens vamos criar objetos `features_column` que transformam os dados para uma forma que pode ser usada por uma RNA.

Os objetos `feature_column` são inseridos na RNA usando uma camada `Densefeature` e os resultados das transformações são as entradas das camadas da rede, sem a necessidade de pré-processamento dos dados.  

Para isso vamos usar um lote de dados de treinamento para exemplificar as transformações.

In [ ]:
# Gera um lote de dados de entrada do conjunto de treinamento
example_batch = next(iter(train_ds))[0]

for exemple_batch in train_ds.take(1):
    for key, value in example_batch.items():
        print("  {}: {}".format(key, value))
    print(' ')

### 4.5 Transformação e preparação dos dados de entrada

O trabalho "Seleção de variáveis e classificação de padrões por redes neurais como auxílio ao diagnóstico de cardipatia isquêmica" (https://www.scielo.br/pdf/pope/v28n2/07.pdf) apresenta as catacrerísticas mais importantes desse conjunto de dados para diagnosticar doença cardiaca. Usando os resultados desse trabalho vamos ignorar as seguintes características:

- "fbs", "sex", "chol" e "trestbps"

As transformações que iremos aplicar em cada uma das características são as seguintes:

- "age": numérico $\to$ segmentação [40, 50, 60, 70]
- "cp": categoria (numérica) $\to$ one-hot
- "restECG": categoria (numérica) $\to$ one-hot
- "thalach": numérico $\to$ normalização entre 0 e 1
- "exang": categoria (0 ou 1)
- "oldpeak": numerico $\to$  segmentação [0.1, 0.81, 1.61]
- "slope": categoria (numérica) $\to$ one-hot
- "ca": categoria (numérica) $\to$ one-hot
- "thal": categoria (string) $\to$ one-hot

Após definir cada uma das transformações desejadas nos dados de entrada temos que preparar esses dados para poderem ser entradas de uma RNA. Para isso temos inicialmente que definir uma lista com as colunas de características que queremos usar.

Vamos definir cada uma das tranformações que queremos realizar nos dados e incluir em uma lista de características que queremos usar como dado de entrada da RNA.

In [ ]:
# Incializa lista de características desejadas
feature_columns_list = []

# Inicializa número de entradas de cada exemplo
num_input = 0

#### Característica "age"

In [ ]:
# Segmentação da coluna numerica "age"
age = tf.feature_column.numeric_column("age")
seg_age = tf.feature_column.bucketized_column(age, boundaries=[40, 50, 60, 70])

# Visulização de um lote
rna_demo(seg_age)

# Inclui seg_age na lista de características desejadas
feature_columns_list.append(seg_age)

# Atualiza número de entradas
num_input += 5

#### Característica "cp"

In [ ]:
# Codificação one-hot dos dados categóricos da coluna "cp"
cp = feature_column.categorical_column_with_vocabulary_list('cp', [0, 1, 2, 3, 4])
cp_hot = tf.feature_column.indicator_column(cp)

# Visualização de um lote
rna_demo(cp_hot)

# Inclui cp_hot na lista de características desejadas
feature_columns_list.append(cp_hot)

# Atualiza número de entradas
num_input += 5

#### Característica "restecg"

In [ ]:
# Codificação one-hot dos dados categóricos da coluna "restecg"
restecg = feature_column.categorical_column_with_vocabulary_list('restecg', [0, 1, 2])
restecg_hot = tf.feature_column.indicator_column(restecg)

# Visualização de um lote
rna_demo(restecg_hot)

# Inclui seg_age na lista de características desejadas
feature_columns_list.append(restecg_hot)

# Atualiza número de entradas
num_input += 3

#### Característica "thalach"

In [ ]:
# Define função de normalização
def norm_max(col):
    return col/max_thalach

# Define valor máximo de "thalach"
max_thalach = 202

# Normalização da coluna "thalach"
thalach_norm = tf.feature_column.numeric_column("thalach", normalizer_fn=norm_max)

# Visualização de um lote 
rna_demo(thalach_norm)

# Inclui thalach_norm na lista de características desejadas
feature_columns_list.append(thalach_norm)

# Atualiza número de entradas
num_input += 1

- Para normalizar dados numéricos temos que definir uma função que realiza essa tranformação e ao criar a coluna numérica devemos chamar essa função.

#### Característica "exang"

In [ ]:
# Codificação dos dados categóricos da coluna "exang"
exang_cat = feature_column.categorical_column_with_vocabulary_list('exang', [0, 1])
exang_cat = tf.feature_column.indicator_column(exang_cat)

# Visualização de um lote
rna_demo(exang_cat)

# Inclui exang_cat na lista de características desejadas
feature_columns_list.append(exang_cat)

# Atualiza número de entradas
num_input += 2

#### Característica "oldpeak"

In [ ]:
# Segmentação da coluna "oldpeak"
oldpeak = tf.feature_column.numeric_column("oldpeak")
seg_oldpeak = tf.feature_column.bucketized_column(oldpeak, boundaries=[0.1, 0.81, 1.61])

# Visualização de um lote
rna_demo(seg_oldpeak)

# Inclui seg_oldpeak na lista de características desejadas
feature_columns_list.append(seg_oldpeak)

# Atualiza número de entradas
num_input += 4

#### Característica "slope"

In [ ]:
# Codificação one-hot dos dados categóricos da coluna "slope"
slope = feature_column.categorical_column_with_vocabulary_list('slope', [1, 2, 3])
slope_hot = tf.feature_column.indicator_column(slope)

# Visualização de um lote
rna_demo(slope_hot)

# Inclui slope_hot na lista de características desejadas
feature_columns_list.append(slope_hot)

# Atualiza número de entradas
num_input += 3

#### Característica "ca"

In [ ]:
# Codificação one-hot dos dados categóricos da coluna "ca"
ca = feature_column.categorical_column_with_vocabulary_list('ca', [0, 1, 2, 3])
ca_hot = tf.feature_column.indicator_column(ca)

# Visualização de um lote
rna_demo(ca_hot)

# Inclui cat_hot na lista de características desejadas
feature_columns_list.append(ca_hot)

# Atualiza número de entradas
num_input += 4

#### Característica "thal"

In [ ]:
# Codificação one-hoe das categorias da coluna "thal"
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_hot = tf.feature_column.indicator_column(thal)

# Visualização de um lote
rna_demo(thal_hot)

# Inclui thal_hot na lista de características desejadas
feature_columns_list.append(thal_hot)

# Atualiza número de entradas
num_input += 3

In [ ]:
print('Numero de entradas de cada exemplo:', num_input)

### 4.6 Criar camada de características (`DenseFeatures`)

Após definir as colunas de características que queremos usar na RNA e as transformações desejadas, temos que criar uma camada do tipo `DenseFeatures` (https://www.tensorflow.org/api_docs/python/tf/keras/layers/DenseFeatures) para receber e transformar as caracteríticas desejadas.

As características desejadas devem estar em uma lista, que no caso é a `feature_columns_list`.

Uma camada do tipo `DenseFeatures` produz um tensor (vetor) baseado nas características e nas transformações definidas na lista de colunas de característica (`feature_columns_list`).

A camada `DenseFeatures` deve ser a primeira camada da RNA após a camada `Input`, se esta existir.  

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns_list)

### 4.7 Definir Dataset com lotes maiores

Anteriormente, usamos um tamanho de lote pequeno para demonstrar como as `feature_column` funcionam. 

Vamos definir  agora um Dataset com um tamanho de lote maior.

In [ ]:
# Define tamanho do lote
batch_size = 32

# Cria Dataset de treinamento
train_ds = df_to_dataset(df_train, batch_size=batch_size)

# Cria Dataset de teste
test_ds = df_to_dataset(df_test, shuffle=False, batch_size=batch_size)

### 4.8 Configuração, compilação e treinamento da RNA

Para resolver esse problema de calcular a probabilidade de um paciente ter doença do coração, vamos utilizar uma RNA simples com três camadas densas. 

Vamos compilar essa RNA vamos com o método de otimização Adams e depois vamos treiná-la usando 50 épocas usando os Datasets de treinamento e teste (`train_ds` e `test_ds`).

Com é um problema de classificação binária, a função de custo mais indicada é a `binary_crossentropy` (função logística). Como métrica vamos utlizar a exatidão (`accuracy`).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

rna = Sequential()
rna.add(feature_layer)
rna.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
rna.add(layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
rna.add(layers.Dense(1, activation='sigmoid'))

rna.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [ ]:
results = rna.fit(train_ds, validation_data=test_ds, epochs=70, verbose=0)

rna.summary()

**Observações:**

- Essa RNA possui 8.129 parâmetros treináveis;

- No sumário da RNA não aprece o número de ativações de cada camada;

- Não é possível usar o método `summary` antes de treinar a RNA porque as dimensões das entradas e ativações das camadas sómente é definida durante o treinamento, quando a RNA recebe os dados de entrada calculados pelas `feature_column`.



#### Mostra gráficos do processo de treinamento

In [ ]:
# Recupera resultados de treinamento do dicinário history
acc      = results.history['accuracy']
val_acc  = results.history['val_accuracy']
loss     = results.history['loss']
val_loss = results.history['val_loss']

# Cria vetor de épocas
epocas   = range(len(acc)) 

# Gráfico dos valores da função de custo
plt.plot(epocas, loss, 'r', label='Custo - treinamento')
plt.plot(epocas, val_loss, 'b', label='Custo - validação')
plt.title('Valor da função de custo – treinamento e validação')
plt.xlabel('Épocas')
plt.ylabel('Custo')
plt.legend()
plt.show()

# Gráfico dos valores da métrica
plt.plot(epocas, acc, 'r', label='exatidao- treinamento')
plt.plot(epocas, val_acc, 'b', label='exatidao - validação')
plt.title('Valor da métrica – treinamento e validação')
plt.xlabel('Épocas')
plt.ylabel('Exatidao')
plt.legend()
plt.show()

### 4.9 Avaliação e teste da RNA

Para verificar o desempenho da RNA vamos calcular o valor da função de custo e da métrica para os dois conjuntos de dados.

In [ ]:
# Avalia desempenho da RNA para os dados de treinamento, validação e teste
eval_train = rna.evaluate(train_ds, verbose=0)
eval_test = rna.evaluate(test_ds, verbose=0)

# Apresenta resultados
print('Dados de treinamento: Função de custo =', eval_train[0], '- Exatidão =', eval_train[1])
print('Dados de teste: Função de custo =', eval_test[0], '- Exatidão =', eval_test[1])

Observa-se que a RNA acerta se o paciente tem doença do coração em cerca de 84% dos casos nos exemplos de teste.

#### Teste da RNA

Para finalizar a avaliação, vamos verificar como a RNA prevê a probabilidade dos pacientes do conjunto de teste terem ou não doença de coração.

In [ ]:
# Gera um exemplos de dados do conjunto de teste
for x, y in test_ds.take(1):
    # Mostra lote gerado
    #print('Entrada:\n', x)

    # calcula previsão da RNA
    y_prev = np.round(rna.predict(x))
    
    # Mostra saída real e prevista
    plt.figure(figsize=(16,4))
    plt.plot(y, 'ro', label='Classe real')
    plt.plot(y_prev, 'bo', label='Classe prevista')
    plt.legend()
    plt.show()